# 3.0 Comparing Models
### After Training your Models, What's Next?
After training multiple models with Anvil, you will want to compare the performance across models in a robust way. We closely follow the guidlines laid out in [this paper](https://chemrxiv.org/engage/chemrxiv/article-details/672a91bd7be152b1d01a926b). Consider the below decision chart for helping figure out which models to compare:

<div style="text-align: center">
<img src="comparison_guidelines.png" alt="Model comparison" width="500"/>.  
</div>

### Requirements
For this demo, you will need:
1. At least 2 models trained with the Anvil workflow.
2. All models trained with same cross validation splits, e.g. 5 splits x 5 repeats
## Overview
This notebook will walk you through how to use the OpenADMET CLI to evaluate models that have been trained with the Anvil workflow. In this particular demo, we will compare the models we trained in `02_Training_Models.ipynb`.

## Use Anvil to Compare Models
As with training models with Anvil, comparing models is also a simple command with the following arguments:
```bash
    openadmet compare \
        --model-stats <path-1/cross_validation_metrics.json> \ # this is the path to the cross_validation_metrics.json file output by anvil of your first model
        --model-tag <a-tag-to-label-your-trained-model-1> \ # this can be any moniker that is distinguishable for you
        --task-name <name-of-task-1> \ # this is the name of your target_cols from the anvil recipe.yaml
        \

        --model-stats <path-2/cross_validation_metrics.json> \ # corresponding info for your second model
        --model-tag <a-tag-to-label-your-trained-model-2> \
        --task-name <name-of-task-2> \

        ... repeat this set of arguments for as many models as you want to compare


        --output-dir <path-to-output-plots> \ # this is an existing directory for your plot to export to
        --report <whether-or-not-to-write-pdf-report>
```
**IMPORTANT NOTE** You can only compare models that have the same number of cross validation folds, e.g. a model with `5 splits x 2 repeats` can only be compared to another model that is also cross validated with `5 splits x 2 repeats`.
For this demo, this command is:

In [ ]:
%%bash
openadmet compare \
    --model-stats ../../02_Model_Training/lgbm/cross_validation_metrics.json \
    --model-tag lgbm \
    --task-name OPENADMET_LOGAC50 \
    --model-stats ../../02_Model_Training/chemprop/cross_validation_metrics.json \
    --model-tag chemprop \
    --task-name OPENADMET_LOGAC50 \
    --model-stats ../../02_Model_Training/multitask/cross_validation_metrics.json \
    --model-tag multitask \
    --task-name OPENADMET_LOGAC50_cyp3a4 \
    --output-dir model_comparisons/ \
    --report True

/Users/cynthiaxu/miniconda3/envs/demos/lib/python3.12/site-packages/openadmet/models/comparison/posthoc.py:476: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '***' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  significance[(hsd.pvalue < self.sig_levels[2]) & (hsd.pvalue >= 0)] = "***"
/Users/cynthiaxu/miniconda3/envs/demos/lib/python3.12/site-packages/openadmet/models/comparison/posthoc.py:477: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '**' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  significance[
/Users/cynthiaxu/miniconda3/envs/demos/lib/python3.12/site-packages/openadmet/models/comparison/posthoc.py:476: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Val

Levene's test results
-------------------------
+------------+-----------+------------+----------+-------------+
|        mse |       mae |         r2 |     ktau |   spearmanr |
|------------+-----------+------------+----------+-------------|
| 5.34346    | 2.76955   | 6.26403    | 1.71125  |    1.02614  |
| 0.00685875 | 0.0693782 | 0.00310412 | 0.187905 |    0.363568 |
+------------+-----------+------------+----------+-------------+

Tukey's HSD results
-------------------------
+-----------------------------+-----------+------------+-------------+-------------+
| method                      | metric    |      value |   errorbars |     p-value |
|-----------------------------+-----------+------------+-------------+-------------|
| lgbm-chemprop               | mse       | -0.0950442 |   0.0480257 | 3.14253e-05 |
| lgbm-chemprop_multitask     | mse       | -0.0244335 |   0.0480257 | 0.446793    |
| chemprop-chemprop_multitask | mse       |  0.0706107 |   0.0480257 | 0.0021588   |
| lgb

Now, in model comparisons, you should find these outputs:
- `Levene.json` - file containing results of Levene test which assesses homogeneity of variances among groups
- `Tukey_HSD.json` - file containing confidence intervals for Tukey HSD (honestly significant difference) test for pairwise comparisons between models
- `anova.pdf` - ANOVA (analsyis of variance) plot showing whether each metric across all the compared models are statistically signficantly different; p-value ≤ 0.05
- `mcs_plots.pdf`- multiple comparisons similarity plot where the color denotes effect size and asterisk annotations denote statistical significance
- `mean_diffs.pdf`- plot of confidence intervals of the difference in mean performance between models; intervals that do not cross the zero line imply statistical significance
- `normality_plots.pdf` - plots to show how normal the distribution of metrics are to check assumptions of parametric tests, e.g. ANOVA, etc.
- `paired_plots.pdf` - plots to check pairwise relationships between metrics across the comparing models
- `posthoc.pdf` - a file containing the tabulated Levene and Tukey HSD results 

### Interpreting the comparison plots

✨✨✨✨✨✨✨